In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import gensim.corpora as corpora
from pprint import pprint
from gensim import  models


import pickle
from gensim.test.utils import datapath

# Import Dataset
papers = pd.read_csv('csv/data.csv')
#print(papers.head())
print(papers.head())

   Unnamed: 0                                               text
0           0  32 THIS ARTICLE HAS BEEN PEER-REVIEWED.  COMPU...
1           1  Interactions of Mean Climate Change and \nClim...
2           2  This journal is cThe Royal Society of Chemistr...
3           3  LETTERS\nPUBLISHED ONLINE: 9 MARCH 2014 | DOI:...
4           4  Del G~nio A. D. (2013) Physical processes cont...


In [2]:
# Remove punctuation
papers['text'] = \
papers['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['text'] = \
papers['text'].map(lambda x: x.lower())
# Print out the first rows of papers
print(papers['text'].head())


stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
             
data = papers.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
/var/folders/cv/d1w1tlhn18d523dngzfhkzcc0000gn/T/ipykernel_3438/1712462136.py:3: DeprecationWarning: invalid escape sequence \.
  papers['text'].map(lambda x: re.sub('[,\.!?]', '', x))


0    32 this article has been peer-reviewed  comput...
1    interactions of mean climate change and \nclim...
2    this journal is cthe royal society of chemistr...
3    letters\npublished online: 9 march 2014 | doi:...
4    del g~nio a d (2013) physical processes contro...
Name: text, dtype: object
['article', 'peer', 'reviewed', 'computing', 'science', 'learning', 'climate', 'informatics', 'accelerating', 'discovering', 'climate', 'science', 'machine', 'learning', 'goal', 'climate', 'informatics', 'emerging', 'discipline', 'inspire', 'collaboration', 'climate', 'scientists', 'data', 'scientists', 'order', 'develop', 'tools', 'analyze', 'complex']


In [3]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])


[(0, 3), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1)]


In [7]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

pprint(lda_model.print_topics())

[(0,
  '0.018*"climate" + 0.015*"data" + 0.011*"change" + 0.010*"yield" + '
  '0.009*"crop" + 0.008*"model" + 0.006*"models" + 0.005*"et" + 0.005*"al" + '
  '0.005*"temperature"'),
 (1,
  '0.017*"climate" + 0.006*"earth" + 0.005*"surface" + 0.005*"al" + 0.005*"et" '
  '+ 0.004*"ocean" + 0.004*"change" + 0.004*"mean" + 0.004*"cloud" + '
  '0.004*"models"'),
 (2,
  '0.017*"climate" + 0.014*"al" + 0.014*"et" + 0.008*"model" + 0.007*"global" '
  '+ 0.006*"ozone" + 0.006*"change" + 0.006*"emissions" + 0.005*"changes" + '
  '0.005*"models"'),
 (3,
  '0.019*"climate" + 0.009*"change" + 0.007*"groundwater" + 0.007*"res" + '
  '0.006*"water" + 0.006*"al" + 0.006*"et" + 0.006*"geophys" + 0.005*"atmos" + '
  '0.005*"changes"'),
 (4,
  '0.017*"climate" + 0.012*"co" + 0.011*"forcing" + 0.010*"change" + '
  '0.009*"temperature" + 0.009*"et" + 0.008*"al" + 0.008*"doi" + '
  '0.008*"sensitivity" + 0.007*"global"')]


In [10]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Users/wenxiuye/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.032493  0.061240       1        1  47.620925
0      0.005182 -0.177406       2        1  17.952596
3     -0.130209 -0.013206       3        1  15.434554
4     -0.043878  0.139173       4        1  13.032934
1      0.136413 -0.009800       5        1   5.958991, topic_info=         Term         Freq        Total Category  logprob  loglift
181   climate  2428.000000  2428.000000  Default  30.0000  30.0000
271      data   742.000000   742.000000  Default  29.0000  29.0000
1643  forcing   496.000000   496.000000  Default  28.0000  28.0000
1521     crop   247.000000   247.000000  Default  27.0000  27.0000
2146    yield   258.000000   258.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
756     model    22.003215   879.929322   Topic5  -5.9397  -0.8684
170   changes    20.107881   629.607917   Topic5  -6.0298  -0.6237
564       ice    18.332356   270.110720   Topic5  -6.1222   0.1301
1142    space    17.053556   152.720765   Topic5  -6.1946   0.6280
1643  forcing    17.752969   496.011126   Topic5  -6.1544  -0.5098

[392 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
7681       1  0.942825        aa
7681       2  0.027730        aa
7681       4  0.027730        aa
5082       1  0.190019  absorbed
5082       4  0.095009  absorbed
...      ...       ...       ...
2156       2  0.972490    zubair
13641      1  0.943378       αch
11516      1  0.041247     ﬁgure
11516      3  0.123742     ﬁgure
11516      4  0.824948     ﬁgure

[798 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 5, 2])